In [1]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from scipy import linalg

In [2]:
def loadVid(path):
	# Create a VideoCapture object and read from input file
	# If the input is the camera, pass 0 instead of the video file name
	cap = cv2.VideoCapture(path)

	# Check if camera opened successfully
	if (cap.isOpened()== False):
		print("Error opening video stream or file")

	i = 0
	# Read until video is completed
	while(cap.isOpened()):
		# Capture frame-by-frame
		i += 1
		ret, frame = cap.read()
		if ret == True:

			#Store the resulting frame
			if i == 1:
				frames = frame[np.newaxis, ...]
			else:
				frame = frame[np.newaxis, ...]
				frames = np.vstack([frames, frame])
				frames = np.squeeze(frames)

		else:
			break

	# When everything done, release the video capture object
	cap.release()

	return frames

In [3]:
img_reference = cv2.imread("Part 1/cv_cover.jpg",0)
frames=loadVid("Part 1/book.mov")
panda_frames=loadVid("Part 1/ar_source.mov")

In [3]:
def overlay(img_reference,book_frame,panda_frame,segma=0.2 , min_correspondences=30):

	sift = cv2.SIFT_create()	#CREATE SIFT OBJECT

	#GET SIFT DESCRIPTORS AND KEYPOINTS FOR COVER IMAGE (REFERENCE)
	kp_reference, des_reference = sift.detectAndCompute(img_reference,None)
	kp_frame, des_frame = sift.detectAndCompute(book_frame,None)

	#MATCH DESCRIPTORS BETWEEN COVER(REFERENCE) AND BOOK_FRAME
	bf = cv2.BFMatcher()
	matches = bf.knnMatch(des_reference,des_frame, k=2)

	#FILTER GOOD CORRESPONDANCES WITH
	# MIN NU OF CORRESPONDENCES = MIN_CORRES  , DISTANCE RATIO BETWEEN 2 MATCHES (KNN) = SEGMA
	# AND GET THEIER COORDINATES
	correspondence_reference = []
	correspondence_frame = []
	while len(correspondence_frame)<min_correspondences:
		correspondence_reference = []
		correspondence_frame = []

		for m,n in matches:
			if m.distance < segma*n.distance:
				correspondence_reference.append(kp_reference[m.queryIdx].pt)
				correspondence_frame.append(kp_frame[m.trainIdx].pt)
		segma=segma+0.005



	#GET A TO SOLVE DLT AH=0
	A = np.empty((0,9))
	for i in range(len(correspondence_reference)):
		u,v=correspondence_frame[i]
		x,y=correspondence_reference[i]
		x=np.array([
				   [-x , -y , -1 , 0 ,0 ,0 ,u*x, u*y , u],
				   [0,0,0,-x,-y,-1,v*x,v*y,v]
				   ])
		A = np.vstack( (A,x) )
	u, s, v = linalg.svd(A)
	DOF=np.reshape(v[-1],(3,3))		#MIN VECTOR

	#ur_pixel=np.matmul(DOF,(img_reference.shape[1],0,1))/np.matmul(DOF,(img_reference.shape[1],0,1))[2]
	#ul_pixel=np.matmul(DOF,(0,0,1))/np.matmul(DOF,(0,0,1))[2]
	#lr_pixel=np.matmul(DOF,(img_reference.shape[1],img_reference.shape[0],1))/np.matmul(DOF,(img_reference.shape[1],img_reference.shape[0],1))[2]
	#ll_pixel=np.matmul(DOF,(0,img_reference.shape[0],1))/np.matmul(DOF,(0,img_reference.shape[0],1))[2]

	#RESIZE ROWS IN PANDA VIDEO FRAME TO FIT ONTO REFERENCE
	#NOTE ROWS VALS (41,44) SELECTED MANUALLY BY OBSERVING THE PANDA_VID FRAMES

	col_to_crop=int((panda_frame.shape[1]-img_reference.shape[1])/2)
	xx=panda_frame[ 44:panda_frame.shape[0]-44 , col_to_crop:panda_frame.shape[1]-col_to_crop, :]
	xx=cv2.resize(xx, (350,440))

	#GET COREESPONDING POINT IN BOOK VIDEO FRAME AND OVERLAY PANDA FRAME ON IT
	temp_frame=book_frame.copy()
	for i in range (xx.shape[0]):
		for j in range (xx.shape[1]):
			try:
				res=np.matmul(DOF,(j,i, 1))
				res=np.ceil(res/res[2]).astype(int)
				if res[0]>=0 and res[1]>=0:
					temp_frame[res[1],res[0],:]=xx[i,j,]
			except:
				return overlay(img_reference,book_frame,panda_frame,min_correspondences=min_correspondences-5)
	#print("segma : ", segma)
	#print("length : ", len(correspondence_frame))
	return temp_frame

In [5]:
#APPLY TRANSFORMATION ON EACH FRAME  PANDA-->BOOK
for i in range(len(panda_frames)):
	img=overlay(img_reference,frames[i].copy(),panda_frames[i].copy(),segma=0.11,min_correspondences=50)
	cv2.imwrite("saved/"+str(i)+".jpg", img)

idle_frames=frames.shape[0]-panda_frames.shape[0]
for i in  range(	panda_frames.shape[0],idle_frames+panda_frames.shape[0]	):
	img=overlay(img_reference,frames[i].copy(),panda_frames[-1].copy(),segma=0.11,min_correspondences=50)
	cv2.imwrite("saved/"+str(i)+".jpg", img)

In [6]:
import os
import cv2

path= "C:\\Users\\moham\\PycharmProjects\\pythonProject\\saved\\"
out_path = "C:\\Users\\moham\\PycharmProjects\\pythonProject\\output_video"
out_video_name= '.mp4'
out_video_full_path=out_path+out_video_name

img = [int(x.split('.')[0]) for x in os.listdir(path)]
img.sort()
img= [str(x)+'.jpg' for x in img]

temp=[]
for i in img:
	temp.append(path+i)

cv2_fourcc=cv2.VideoWriter_fourcc(*'mp4v')

frame=cv2.imread(img[0])
size=list(frames[0].shape)
del size[2]
size.reverse()

video = cv2.VideoWriter(out_video_full_path,cv2_fourcc,30,size)

for i in range (len(temp)):
	video.write(cv2.imread(temp[i]))
video.release()